# **COVID-19 Data Visualization Dashboard**

#Objective:
>To apply basic data science techniques including data cleaning, exploration, visualization, and simple modeling using Python.

**Setup Libraries**

**Purpose:**

Install all required Python libraries (pandas, plotly, streamlit) and NodeJS package for LocalTunnel (if needed).

Import all necessary packages for data manipulation, visualization, and dashboard creation.

In [ ]:
!pip install pandas plotly streamlit pyngrok --quiet

import pandas as pd
import plotly.express as px
from pyngrok import ngrok
import streamlit as st


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.0 MB/s eta 0:00:00


#**Load Datase**

#Purpose:
#Download COVID-19 dataset from Our World in Data (stable GitHub link)
#Load it into a pandas DataFrame


#**Data Cleaning and Preprocessing**

#Purpose:
#Keep only important columns: location, date, total_cases, new_cases, total_deaths, new_deaths, population
#Convert date column to datetime
#Remove rows with missing total_cases

In [ ]:
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"

df = pd.read_csv(url)
df.head()


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.75,0.5,64.83,0.51,41128772,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.75,0.5,64.83,0.51,41128772,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-07,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.75,0.5,64.83,0.51,41128772,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-08,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.75,0.5,64.83,0.51,41128772,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-09,0.0,0.0,NaN,0.0,0.0,NaN,...,NaN,37.75,0.5,64.83,0.51,41128772,NaN,NaN,NaN,NaN



#**Test Visualization **

#Purpose:
#Filter dataset for a single country
#Create a simple interactive line chart of total COVID-19 cases

In [ ]:
# Select useful columns
df = df[['location', 'date', 'total_cases', 'new_cases',
         'total_deaths', 'new_deaths', 'population']]

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Remove rows with missing case numbers
df = df.dropna(subset=['total_cases'])

# Show cleaned data
df.head()


,location,date,total_cases,new_cases,total_deaths,new_deaths,population
0,Afghanistan,2020-01-05,0.0,0.0,0.0,0.0,41128772
1,Afghanistan,2020-01-06,0.0,0.0,0.0,0.0,41128772
2,Afghanistan,2020-01-07,0.0,0.0,0.0,0.0,41128772
3,Afghanistan,2020-01-08,0.0,0.0,0.0,0.0,41128772
4,Afghanistan,2020-01-09,0.0,0.0,0.0,0.0,41128772


#**Create Streamlit Dashboard**

#Purpose:
#Write the main dashboard file app.py
#Include:
#Country selector
#Total cases graph
#New daily cases graph
#Total deaths graph

In [ ]:
# Filter for India (you can change to any country)
india_df = df[df['location'] == 'India']

# Plot total COVID-19 cases over time
fig = px.line(india_df,
              x='date',
              y='total_cases',
              title='India - Total COVID-19 Cases Over Time')

fig.show()


#**Run Streamlit Dashboard **
#Purpose:
#Run the Streamlit app in the background
#Use Cloudflare Tunnel to get a stable, password-free public URL

In [ ]:
%%writefile app.py

import pandas as pd
import plotly.express as px
import streamlit as st

# Load dataset
url = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
df = pd.read_csv(url)

# Select useful columns
df = df[['location', 'date', 'total_cases', 'new_cases',
         'total_deaths', 'new_deaths', 'population']]

# Convert date
df['date'] = pd.to_datetime(df['date'])

# Drop missing values for main metrics
df = df.dropna(subset=['total_cases'])

# Title
st.title("COVID-19 Interactive Dashboard")
st.write("Explore COVID-19 trends across different countries")

# Country dropdown
countries = df['location'].unique()
country = st.selectbox("Select a Country", sorted(countries))

# Filter for selected country
filtered = df[df['location'] == country]

# Total Cases Plot
fig1 = px.line(filtered, x='date', y='total_cases',
               title=f"{country} - Total Cases Over Time")
st.plotly_chart(fig1)

# New Daily Cases
fig2 = px.line(filtered, x='date', y='new_cases',
               title=f"{country} - New Daily Cases")
st.plotly_chart(fig2)

# Total Deaths
fig3 = px.line(filtered, x='date', y='total_deaths',
               title=f"{country} - Total Deaths Over Time")
st.plotly_chart(fig3)


Writing app.py


#**Install LocalTunnel**

#Purpose:

#Install the LocalTunnel npm package globally.

#LocalTunnel allows your Streamlit dashboard to be exposed publicly via a temporary URL.

#This step is necessary to share your dashboard with others without deploying to a cloud server.


#Notes:

#Requires NodeJS to be installed (Colab usually has it pre-installed).






In [ ]:
# Install localtunnel
!npm install -g localtunnel

# Run Streamlit app in background
!streamlit run app.py &>/dev/null &

# Generate a public URL
!lt --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
changed 22 packages in 2s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴your url is: https://busy-colts-sip.loca.lt
^C


#**Install Cloudflare Tunnel (cloudflared)**

#Purpose:

#Download and install the Cloudflare Tunnel client (cloudflared) on the Colab environment.

#cloudflared allows you to expose your local Streamlit app to a public, stable, password-free URL.

#Why use Cloudflare Tunnel:

#More reliable than LocalTunnel or ngrok in Colab.

#No authentication errors or temporary password prompts.

#Provides a stable URL that can be shared with anyone to view your dashboard.

#Notes:
#This is a system-level installation, so it only needs to be done once per Colab session.
#After installation, you can use cloudflared tunnel --url http://localhost:8501 to get the live dashboard link.

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


--2025-11-30 16:05:27--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2025-11-30 16:05:27--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/8a32f7c6-649c-4f0d-806d-e14c19d0786d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-30T16%3A56%3A13Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4d

#Run Streamlit App in Background

#Purpose:

#Launch your Streamlit dashboard (app.py) in the background on port 8501.

#Prepares the app so it can be accessed by a public URL via Cloudflare Tunnel.

#Why:

#Running Streamlit in the background prevents the Colab notebook from freezing.

#Keeps the app active so the tunnel can connect to it and generate a live URL.


In [ ]:
!streamlit run app.py &>/dev/null &


#Start Cloudflare Tunnel

#Purpose:

#Create a public, stable URL that exposes your local Streamlit app (app.py) to the internet.

#Makes your dashboard accessible from any browser, without needing Colab to stay open in the foreground.

#Why:

#Cloudflare Tunnel provides a reliable, password-free, temporary URL.

#Unlike LocalTunnel or ngrok, it avoids frequent connection errors or authentication issues.

#Enables easy sharing of this COVID-19 dashboard for portfolio or demonstration purposes.

In [ ]:
!cloudflared tunnel --url http://localhost:8501


2025-11-30T16:47:05Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-30T16:47:05Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-30T16:47:08Z INF +--------------------------------------------------------------------------------------------+
2025-11-30T16:47:08Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-30T16:47:08Z INF |  https://display-brothers-wild-adam.trycloudflare.com 